# Step 1: Load Images

In [1]:
from PIL import Image
import os

def load_image(image_path):
    """
    Load an image from the given path and return it.
    """
    return Image.open(image_path).convert('RGB')


# Step 2: Preprocess Images

In [11]:
import torchvision.transforms as transforms

def preprocess_image(image):
    """
    Preprocess the given image to the format required by the model.
    """
    
    # Define the image transformations (resize, center crop, and normalization)
    preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    
    return preprocess(image)


In [21]:
image1_tensor = preprocess_image(load_image(r"000000166598.jpg"))
image2_tensor = preprocess_image(load_image("000000414738.jpg"))

# Part 3: Extract Embeddings

In [12]:
import torch
import torchvision.models as models

# Load a pre-trained ResNet model
model = models.resnet50(pretrained=True)
# Remove the final classification layer
model = torch.nn.Sequential(*(list(model.children())[:-1]))

# Set the model to evaluation mode
model.eval()

def extract_embedding(image_tensor):
    """
    Extract the embedding of an image tensor using the pre-trained model.
    """
    with torch.no_grad():
        embedding = model(image_tensor.unsqueeze(0))
    return embedding.squeeze()


c:\Users\ktumm\anaconda3\envs\big_data\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ktumm\anaconda3\envs\big_data\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [22]:
embedding2 = extract_embedding(image2_tensor)
embedding1 = extract_embedding(image1_tensor)

# Step 4: Compute Similarity

In [20]:
import torch.nn.functional as F

def compute_cosine_similarity(embedding1, embedding2):
    """
    Compute the cosine similarity between two embeddings.
    """
    return F.cosine_similarity(embedding1, embedding2, dim=0)


In [23]:
compute_cosine_similarity(embedding1, embedding2)

tensor(0.6895)

In [14]:
def image_similarity(image_path1, image_path2):
    """
    Compute the similarity between two images given their paths.
    """
    # Load and preprocess images
    image1 = load_image(image_path1)
    image2 = load_image(image_path2)
    
    image1 = preprocess_image(image1)
    image2 = preprocess_image(image2)
    
    # Extract embeddings
    embedding1 = extract_embedding(image1)
    embedding2 = extract_embedding(image2)
    
    # Compute similarity
    similarity = compute_cosine_similarity(embedding1, embedding2)
    
    return similarity.item()

# Example usage:
# similarity = image_similarity("path/to/image1.jpg", "path/to/image2.jpg")
# print(f"Similarity: {similarity}")


In [15]:
image_similarity("000000166598.jpg", "000000414738.jpg")


0.6895161271095276